In [1]:
!pip install tensorflow==2.3.0
!pip install gym
!pip install keras
!pip install keras-rl2

You should consider upgrading via the 'c:\users\sheza\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\sheza\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\sheza\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\sheza\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


# 1. Test Random Environment with OpenAI Gym

In [2]:
import gym 
import random 

In [3]:

env = gym.make('CartPole-v0') #Build cartpole environment
states  = env.observation_space.shape[0] #extract the states in the environment - 4 
actions = env.action_space.n #Extract actions from environment - which is 2 left or right

States:
Num     Observation               Min                     Max
0       Cart Position             -4.8                    4.8
1       Cart Velocity             -Inf                    Inf
2       Pole Angle                -0.418 rad (-24 deg)    0.418 rad (24 deg)
3       Pole Angular Velocity     -Inf                    Inf

Actions:
Num   Action
0     Push cart to the left
1     Push cart to the right

In [4]:
actions

2

In [5]:
#We make an environment without deep learning 

episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()    # First we render our environment
        action = random.choice([0,1]) # We take a random step/action - 0 = left and 1 = right
        n_state, reward, done, info = env.step(action) # We then apply that action to our environment and we get some info from this
        # Done means if the game has been complted or not - game is fail if pole is more than degrees
        score+=reward # Based on the steps we get a reward - max score is 200
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:26.0
Episode:2 Score:13.0
Episode:3 Score:24.0
Episode:4 Score:12.0
Episode:5 Score:41.0
Episode:6 Score:41.0
Episode:7 Score:46.0
Episode:8 Score:16.0
Episode:9 Score:10.0
Episode:10 Score:41.0


# 2. Create a Deep Learning Model with Keras

In [6]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [19]:
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1,states))) # Pass in the states
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear')) # Pass in actions - which ones to take
    return model

In [20]:
model = build_model(states, actions)

In [21]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 4)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 24)                120       
_________________________________________________________________
dense_4 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 50        
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


# 3. Build Agent with Keras-RL

In [10]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [11]:
def build_agent(model, actions): # Pass throught themodel we created earlier and the different actions you can take
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                   nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [22]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
    1/10000 [..............................] - ETA: 15:57 - reward: 1.0000

c:\users\sheza\appdata\local\programs\python\python37\lib\site-packages\rl\memory.py:40: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 89s 9ms/step - reward: 1.0000
102 episodes - episode_reward: 97.324 [10.000, 200.000] - loss: 3.264 - mae: 19.542 - mean_q: 39.569

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 85s 9ms/step - reward: 1.0000
55 episodes - episode_reward: 182.636 [133.000, 200.000] - loss: 5.841 - mae: 38.989 - mean_q: 78.440

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 89s 9ms/step - reward: 1.0000
51 episodes - episode_reward: 196.353 [172.000, 200.000] - loss: 5.281 - mae: 41.141 - mean_q: 82.712

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 89s 9ms/step - reward: 1.0000: 0s - rewar
50 episodes - episode_reward: 197.960 [150.000, 200.000] - loss: 4.435 - mae: 39.653 - mean_q: 79.775

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 105s 11ms/step - reward: 1.0000
done, took 457.137 seconds


In [23]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200
Episode 11: reward: 200.000, steps: 200
Episode 12: reward: 200.000, steps: 200
Episode 13: reward: 200.000, steps: 200
Episode 14: reward: 200.000, steps: 200
Episode 15: reward: 200.000, steps: 200
Episode 16: reward: 200.000, steps: 200
Episode 17: reward: 200.000, steps: 200
Episode 18: reward: 200.000, steps: 200
Episode 19: reward: 200.000, steps: 200
Episode 20: reward: 200.000, steps: 200
Episode 21: reward: 200.000, steps: 200
Episode 22: reward: 200.000, steps: 200
Episode 23: reward: 200.000, steps: 200
Episode 24: reward: 200.000, steps: 200
Episode 25: reward: 

In [25]:
_ = dqn.test(env, nb_episodes=15, visualize=True)

Testing for 15 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200
Episode 11: reward: 200.000, steps: 200
Episode 12: reward: 200.000, steps: 200
Episode 13: reward: 200.000, steps: 200
Episode 14: reward: 200.000, steps: 200
Episode 15: reward: 200.000, steps: 200


In [26]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)

In [27]:
del model
del dqn
del env

In [28]:
env = gym.make('CartPole-v0')
actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [29]:
dqn.load_weights('dqn_weights.h5f')

In [30]:
_ = dqn.test(env, nb_episodes=5, visualize=True)

Testing for 5 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
